In [21]:
import pandas as pd
import re
import unidecode

In [22]:
df = pd.read_excel('DuLieuLienHe.xlsx')
df.head(3)

,Tên,Đường,Mã số thuế,Điện thoại,Diđộng,lọc di động,Email,Thành phố,Quốc gia,Là một công ty,...,Loại khách hàng,Map thẻ với off,Unnamed: 15,Tên hiển thị,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,CÔNG TY CỔ PHẦN VNG,"Z06 Đường số 13, Phường Tân Thuận Đông, Quận 7...",0303490096,(028) 39623888,2839623888,2839623888,none@gmail.com,TP Hồ Chí Minh (VN),Việt Nam,True,...,NaN,KHÁCH QUẢNG CÁO LANDINGPAGE,NaN,CTY TƯ VẤN,NaN,NaN,NaN,NaN,NaN,NaN
1,CÔNG TY TNHH TOYO PIANO VIỆT NAM,"Xưởng: Lô I - 10A, đường số 13, KCN Tân Bình, ...",0303277138,+84 28 3816 3692/ 02838163691,0919805757,0919805757,khanhtruc@toyo-piano.com.vn,TP Hồ Chí Minh (VN),Việt Nam,True,...,NaN,NaN,NaN,KHÁCH ANH TRUNG KT,NaN,NaN,NaN,NaN,NaN,NaN
2,CÔNG TY TNHH SANDENKI VIỆT NAM,"Xưởng số T29-1, Thửa đất số 260, Đường số 7, K...",3702258541,+84 274 3800 432,0919805757,0919805757,sdgthanh@gmail.com,Bình Dương (VN),Việt Nam,True,...,NaN,NaN,NaN,KHÁCH CHỊ HÂN KĐAT3,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.columns

Index(['Tên', 'Đường', 'Mã số thuế', 'Điện thoại', 'Diđộng', 'lọc di động',
       'Email', 'Thành phố', 'Quốc gia', 'Là một công ty', 'MAP NVKD',
       'Nhân viên kinh doanh', 'ID bên ngoài', 'Loại khách hàng',
       'Map thẻ với off', 'Unnamed: 15', 'Tên hiển thị', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22'],
      dtype='object')

In [24]:
# Drop các cột có tên bắt đầu với "Unnamed"
df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

# Kiểm tra lại các cột sau khi drop
print(df.columns)


Index(['Tên', 'Đường', 'Mã số thuế', 'Điện thoại', 'Diđộng', 'lọc di động',
       'Email', 'Thành phố', 'Quốc gia', 'Là một công ty', 'MAP NVKD',
       'Nhân viên kinh doanh', 'ID bên ngoài', 'Loại khách hàng',
       'Map thẻ với off', 'Tên hiển thị'],
      dtype='object')


In [25]:

# Hàm kiểm tra email hợp lệ
def is_valid_email(email):
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return bool(re.match(pattern, str(email)))

def remove_accents(input_str):
    """Loại bỏ dấu tiếng Việt khỏi chuỗi."""
    return unidecode.unidecode(input_str)

def clean_and_normalize_email(email, company_name):
    """Chuẩn hóa email bằng cách loại bỏ dấu, khoảng trắng, ký tự không hợp lệ và giữ lại email đầu tiên nếu có nhiều email."""
    if pd.isna(email) or not email.strip():
        clean_name = remove_accents(str(company_name).strip().replace(" ", "").lower())
        return f"{clean_name}@default.com"
    
    # Cắt email thành danh sách và chọn email đầu tiên
    emails = re.split(r'[;,\s]+', email.strip())
    cleaned_email = emails[0] if emails else None
    
    if is_valid_email(cleaned_email):
        return cleaned_email  # Nếu hợp lệ, giữ nguyên
    
    # Nếu email không hợp lệ, tạo email mới từ tên không dấu
    clean_name = remove_accents(str(company_name).strip()).lower()
    return f"{clean_name}@default.com"

# Áp dụng xử lý email cho từng dòng trong DataFrame

df_valid = df[df["Email"].apply(is_valid_email)]
df_invalid = df[~df["Email"].apply(is_valid_email)]
df_invalid["Email"] = df_invalid.apply(lambda row: clean_and_normalize_email(row["Email"], row["Tên"]), axis=1)
df_combined = pd.concat([df_valid, df_invalid]).sort_index()

C:\Users\DanHuy\AppData\Local\Temp\ipykernel_16140\1783937245.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_invalid["Email"] = df_invalid.apply(lambda row: clean_and_normalize_email(row["Email"], row["Tên"]), axis=1)


In [26]:
df_valid.to_excel('ValidData.xlsx', index=False, engine="openpyxl")
df_invalid.to_excel('invalidData.xlsx', index=False, engine="openpyxl")
df_combined.to_excel('cleanedData.xlsx', index=False, engine="openpyxl")